In [4]:
# imports

from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [5]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [6]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [7]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [8]:
# prompts

system_prompt = "You are a helpful technical tutor who answers questions about python code, software engineering, data science and LLMs"
user_prompt = "Please give a detailed explanation to the following question: " + question

In [9]:
# messages

messages=[
    {"role":"system","content":system_prompt},
    {"role":"user","content":user_prompt}
]

In [10]:
# Get gpt-4o-mini to answer, with streaming

stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages,stream=True)
    
response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

The provided code snippet is a Python expression that uses the `yield from` statement along with a set comprehension to generate a sequence of authors from a collection of books. Let’s break down the components of this code to understand what it does and why it may be structured this way.

### Breakdown of the Code

1. **Set Comprehension**: 
   python
   {book.get("author") for book in books if book.get("author")}
   
   This part of the code is a set comprehension. A set is a collection of unique elements in Python. The expression iterates over a collection named `books`, which we can assume is likely a list or another iterable structure containing dictionaries (since the code uses `book.get(...)`).

   - Inside the comprehension, it iterates over each `book` in `books`.
   - The expression `book.get("author")` tries to retrieve the value associated with the key "author" from each `book` dictionary.
   - The condition `if book.get("author")` ensures that only books with a non-None, non-empty author value are included in the resulting set. If a book does not have an author or if the author is falsy (like `None` or an empty string), it is excluded.

   The result is a set containing unique author names from the provided `books` collection.

2. **Yield from**: 
   python
   yield from ...
   
   The `yield from` statement is used inside a generator function to yield all values from an iterable (in this case, the set created by the comprehension). When a generator function is called, it returns a generator object without starting execution immediately. The execution begins when the generator is iterated over, and each time a value is yielded, the generator can pause its state.

   By using `yield from`, the generator will produce each author value sequentially. If the set comprehension yields a set of authors, `yield from` will iterate over that set and yield each author back to whoever is consuming the generator.

### Why Use This Structure?

1. **Functionality**: The main purpose of this code is to obtain and yield unique authors from a set of book items. The set comprehension ensures that there are no duplicate authors in the result.

2. **Generator Behavior**: Using `yield from` makes the code concise and allows for a lazy evaluation of the authors. This is efficient, especially if the list of books is large, as authors are only generated when requested.

3. **Set for Uniqueness**: Using a set directly means duplicate author names are automatically filtered out, which can be useful if `books` potentially contains multiple entries with the same author.

### Final Thoughts

Assuming this code is encapsulated inside a generator function, when you call this generator, it will produce a sequence of unique authors from the provided `books`. The use of both yield and set comprehension here is effective for cases where uniqueness of elements is desired, and where memory efficiency is a priority due to the potential size of data. 

Here's how you might see this in context:

python
def unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}


In this context, `unique_authors` will yield each unique author found in `books`, and you can use it like so:

python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": "Author A"},
    {"title": "Book 4"}  # No author
]

for author in unique_authors(books):
    print(author)


This would output:

Author A
Author B


With the use of a generator, you can efficiently handle potentially large datasets without needing to store the entire result set in memory at once.

In [11]:
# Get Llama 3.2 to answer

response = ollama.chat(model=MODEL_LLAMA, messages=messages)
reply = response['message']['content']
display(Markdown(reply))

Let's break down this line of code.

**What is this code doing?**

This code is using a combination of Python features to extract a list of authors from a collection of books, while avoiding certain issues with iteration.

Here's what's happening:

1. `yield from`: This keyword is used in Python 3.3 and later versions. It's short for "yield from" or "pass through," which means that it sends each item yielded by the inner generator expression to the outer generator.
2. `{book.get("author") for book in books if book.get("author")}`: This is a generator expression, also known as a "list comprehension with generators." It's equivalent to writing a regular list comprehension but uses generators instead of lists.

Here's what this part does:

* `for book in books`: Iterates over the `books` collection.
* `if book.get("author")`: Filters out any book that doesn't have an "author" key. If a book is missing this key, it will be skipped in the iteration.
* `book.get("author")`: Retrieves the value associated with the "author" key from each book dictionary.

So, putting it all together, `yield from {book.get("author") for book in books if book.get("author")}`:

1. Iterates over each book in the `books` collection.
2. Skips any book without an "author" key.
3. Yields (i.e., sends) the author's name from each remaining book dictionary.

**Why is this code necessary?**

In Python, when you use a list comprehension or a for loop with a generator expression, it creates a temporary container (a list or iterator object) that stores the results of the expression. This can lead to memory issues if the collection being iterated over is very large.

By using `yield from` and a generator expression, this code avoids creating an intermediate list or iterator object. Instead, it directly yields each author's name one by one, which is more memory-efficient for large collections.

This technique is commonly used when working with large datasets, as it helps reduce memory usage and prevents potential out-of-memory errors.

**Example Use Case**

Suppose you have a collection of book dictionaries, like this:
```python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3"},  # missing author key
    {"title": "Book 4", "author": "Author D"}
]
```
Using the given code, you can extract a list of authors like this:
```python
authors = yield from {book.get("author") for book in books if book.get("author")}
print(authors)  # Output: ["Author A", "Author B", "Author D"]
```
This will print out the names of all authors, without creating an intermediate list or iterator object.

In [12]:
def input_question(question):
    system_prompt = "You are a helpful technical tutor who answers questions about python code, software engineering, data science and LLMs"
    user_prompt = f" Please give a detailed explanation for this question: {question}"

    # messages

    messages=[
        {"role":"system","content":system_prompt},
        {"role":"user","content":user_prompt}
    ]

    print("Fetching details of GPT-4o")
    # Get gpt-4o-mini to answer, with streaming

    stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages,stream=True)
        
    response_GPT = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response_GPT += chunk.choices[0].delta.content or ''
        response_GPT = response_GPT.replace("```","").replace("markdown", "")
        update_display(Markdown(response_GPT), display_id=display_handle.display_id)

    print("Fetching response from Llama 3.2...")
    response_llama = ollama.chat(model=MODEL_LLAMA, messages=messages)
    reply_llama = response_llama['message']['content']
    display(Markdown(reply_llama))
    

In [13]:
your_question = input("Please enter your question: ")
input_question(your_question)

Please enter your question:  what are LLms and its applications


Fetching details of GPT-4o


Certainly! Let's break down the concept of LLMs, which stands for "Large Language Models."

### What are LLMs?

Large Language Models are a class of artificial intelligence (AI) systems that use machine learning techniques to understand and generate human-like text based on the data they were trained on. These models are built primarily using neural networks, particularly a type called transformers, which have become the state-of-the-art architecture for various natural language processing (NLP) tasks.

#### Key Features of LLMs:

1. **Scale**: LLMs are named for their size—the "large" in LLM refers to the number of parameters (weights) in the model, which can range from millions to hundreds of billions. Training these models requires vast amounts of text data and significant computational resources.

2. **Contextual Understanding**: LLMs can capture contextual relationships between words and phrases, enabling them to understand nuances in language better than smaller models.

3. **Pre-training and Fine-tuning**:
   - **Pre-training**: LLMs are initially trained on a large corpus of text in an unsupervised manner, allowing them to learn grammar, facts, and the structure of the language without specific task instructions.
   - **Fine-tuning**: After pre-training, these models can be fine-tuned on specific datasets to optimize their performance for particular applications.

4. **Generative Capabilities**: Many LLMs can generate coherent and contextually relevant text, making them useful for a wide array of applications beyond mere text classification or sentiment analysis.

### Applications of LLMs:

LLMs have a broad range of applications across various domains. Here are some of the most notable ones:

1. **Chatbots and Virtual Assistants**: LLMs can power conversational agents that assist users in customer service, FAQs, and more, providing human-like responses.

2. **Content Creation**: They can generate articles, blogs, stories, and even poetry. This functionality can be utilized in marketing, social media, and entertainment industries.

3. **Translation Services**: LLMs can help in translating text between languages, providing not just word-for-word translations but contextually relevant interpretations.

4. **Text Summarization**: They can condense longer text documents into summaries, extracting key points while maintaining coherence.

5. **Sentiment Analysis**: Businesses use LLMs for analyzing customer feedback, reviews, or social media sentiment to drive decision-making.

6. **Code Generation**: Some LLMs are trained on programming languages and can assist in code completions or generate functional code snippets, aiding software developers.

7. **Education**: LLMs can assist in generating educational material, answering student queries, grading essays, and providing personalized learning experiences.

8. **Healthcare**: They can process medical literature, assist in diagnosing by analyzing patient records, and help healthcare providers communicate more effectively with patients.

9. **Legal Services**: LLMs can assist lawyers by summarizing cases, drafting documents, and conducting legal research by analyzing vast amounts of legal texts.

10. **Creative Writing**: In art and entertainment, LLMs can collaborate with writers and artists to brainstorm ideas, generate scripts, or create interactive storytelling experiences.

### Challenges and Considerations:

While LLMs have significant potential, they also pose challenges:

- **Bias**: LLMs can inadvertently perpetuate biases present in their training data, leading to unfair or prejudiced outcomes.
  
- **Misinformation**: They might generate text that sounds plausible but is factually incorrect, necessitating rigorous fact-checking.

- **Ethical Concerns**: The use of LLMs can raise questions about authorship, content ownership, and the implications of AI in society.

- **Resource Intensive**: Training and running LLMs require substantial computational resources, raising environmental and accessibility concerns.

In summary, LLMs represent a transformative advancement in AI and natural language processing, offering a range of applications that can significantly enhance human capabilities across various fields. As the technology continues to evolve, addressing its challenges will be crucial for its responsible and effective use.

Fetching response from Llama 3.2...


**What is an LLM (Large Language Model)?**

A Large Language Model (LLM) is a type of artificial intelligence (AI) model that is designed to process and understand human language. These models are trained on vast amounts of text data, which enables them to learn patterns, relationships, and nuances of language.

LLMs are typically based on deep learning architectures, such as transformer models, which use self-attention mechanisms to weigh the importance of different words or tokens in a sentence. This allows LLMs to capture contextual information and generate human-like language outputs.

**How do LLMs work?**

The training process for an LLM involves the following steps:

1. **Data collection**: A massive corpus of text data is collected, which can come from various sources such as books, articles, conversations, or even social media posts.
2. **Preprocessing**: The text data is preprocessed to remove irrelevant information, such as punctuation, special characters, and stop words (common words like "the", "and", etc.).
3. **Model training**: The LLM is trained on the preprocessed data using a supervised or unsupervised learning approach. The goal is to predict the next word in a sequence, given the context of the previous words.
4. **Model evaluation**: The trained model is evaluated on a validation set to assess its performance and identify areas for improvement.

**Applications of LLMs**

LLMs have numerous applications across various industries:

1. **Language Translation**: LLMs can be used for machine translation, where they can translate text from one language to another with high accuracy.
2. **Text Summarization**: LLMs can summarize long documents or articles into concise summaries, highlighting the main points and key information.
3. **Question Answering**: LLMs can answer questions based on the input text, providing relevant and accurate responses.
4. **Sentiment Analysis**: LLMs can analyze text to determine its sentiment, emotional tone, or polarity (positive/negative).
5. **Chatbots and Virtual Assistants**: LLMs can power chatbots and virtual assistants, enabling them to understand user queries and respond accordingly.
6. **Content Generation**: LLMs can generate human-like content, such as news articles, product descriptions, or even entire books.
7. **Natural Language Processing (NLP)**: LLMs are used in various NLP tasks, including text classification, entity recognition, and language modeling.
8. **Speech Recognition**: LLMs can improve speech recognition systems by better understanding the context and nuances of human speech.

**Real-world examples of LLMs**

Some notable examples of LLMs include:

1. **Google's BERT**: A pre-trained language model that has achieved state-of-the-art results in various NLP tasks.
2. **Facebook's M** : A language model used for chatbots and virtual assistants on Facebook platforms.
3. **Microsoft's Turing-NLG**: A natural language generation (NLG) model developed by Microsoft to generate human-like content.

In conclusion, LLMs are powerful AI models that have revolutionized the field of NLP. Their applications are vast and diverse, ranging from simple text processing tasks to complex content generation and language translation.

Do you have any specific questions about LLMs or their applications?